In [1]:
!pip install openai -q

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key="your_dummy_or_real_key",
    base_url="http://10.0.38.50:50019/gemma-4b-q8/v1"
)

response = client.chat.completions.create(
    model="gemma-3-4b-it-Q8_0",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, test the model."}
    ]
)

print(response.choices[0].message.content)


Hello there! 😊 

Okay, let’s test it out! I’m ready. 

What would you like to do? Do you want me to:

*   Answer a question?
*   Write something (a poem, story, email, etc.)?
*   Translate something?
*   Summarize a text?
*   Just chat?

Let me know!


In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field

client = OpenAI(
    api_key="your_dummy_or_real_key",
    #base_url="http://10.0.38.50:50019/gemma-4b-q8/v1"
    base_url="http://192.168.159.92:8585/v1"
)

class OCRCleanedText(BaseModel):
    text: str = Field(..., description="Cleaned and consolidated OCR text")

def clean_ocr_text(*ocr_outputs: str) -> OCRCleanedText:
    formatted_outputs = "\n".join(
        f"-----\nOCR output {i+1}:\n{txt}\n-----" 
        for i, txt in enumerate(ocr_outputs)
    )

    messages = [
        {
            "role": "system",
            "content": (
                "You are a Persian and English text rewriter. "
                "You will get OCR outputs separated like this (all for the same document, from different models):\n"
                "-----\nOCR output 1:\n...\n-----\nOCR output 2:\n...\n-----\n"
                "Use OCR 1 as the main template. Rewrite it to fix wrong words and obvious OCR mistakes. "
                "Use other outputs only if they suggest a better word for corrupted parts in OCR 1. "
                "Do not add extra words. Avoid repeating content. "
                "Respond only in JSON: {\"text\": \"...\"}."
            )
        },
        {
            "role": "user",
            "content": formatted_outputs
        }
    ]

    completion = client.chat.completions.parse(
        model="gemma-3-4b-it-Q8_0",
        messages=messages,
        response_format=OCRCleanedText
    )

    return completion.choices[0].message.parsed


In [31]:
ocr1 = """امروز هواش خوب بود و من به پارک رفتم.
کتاب جدیدی خریدمر و شروع به خواندن آن کردم.
این تجربه خذیلی لذت‌بخش بود!"""

ocr2 = """امروز هوا خوب بود و من به پارک رفتم
کتاب جدید خریدم و شروع به خواندن آن کردم.
این تجربه خیلی لذت بخش بود"""

ocr3 = """امروز هوا خوب بود و من به پارک رفتم.
کتاب جدیدی خریدم و شروع به خواندنش کردم.
این تجربه خیلی لذت بخش بود!"""

result = clean_ocr_text(ocr1, ocr2, ocr3)
print(result.text)


امروز هواش خوب بود و من به پارک رفتم.
کتاب جدیدی خریدم و شروع به خواندن آن کردم.
این تجربه خیلی لذت‌بخش بود!
